In [10]:
with open("input.txt", 'r') as rfile:
	a,b = rfile.readline().strip().split()
print(a,b)
with open("output.txt", 'w') as wfile:
	wfile.write(str(int(a) + int(b)))

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

In [1]:
import numpy as np
import math
def looking(matrix):
    return max(map(max, matrix))

In [12]:
matrix = np.zeros((2,2))
matrix[0][0] = -1
matrix[1][1] = -5
matrix[1][1] = -5
matrix[1][1] = -5
print(looking(matrix))

0.0


In [13]:
a = 2
matrix = np.array([ [0,0,0], [0,0,0], [0,0,0] ])
matrix[0][0] = 1
matrix[0][1] = -a
matrix[1][0] = a**2
print(np.shape(matrix))
print(np.abs(matrix))

(3, 3)
[[1 2 0]
 [4 0 0]
 [0 0 0]]


In [14]:
A = np.array([[1,0,0],[0,1,0],[0,0,1]])
W = np.array([[1,4,5],[2,5,5],[3,6,5]])
print(len(W[0]))
print(W[0])
R = np.zeros((3, 2))
R[:,0] = A.dot(W[:,0])
for i in range(2):
    R[:,i] = A.dot(W[:,i])
print(R)

3
[1 4 5]
[[1. 4.]
 [2. 5.]
 [3. 6.]]


In [55]:
a = np.array([2,2,2])
b = np.array([2,2,2])
print(a / b)


[1. 1. 1.]


In [22]:
matrix = np.zeros((4,8))
matrix[1][1] = 19
print(matrix)

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 19.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [4]:
for i in range(11,-1,-1):
    print(i)

11
10
9
8
7
6
5
4
3
2
1
0


In [8]:
def abs(number):
    return number if number >= 0 else -number
print(abs(-5))

5


In [8]:
a = np.array([1,2,3])
a = np.vstack((a,[1,4,4])
print(a)

SyntaxError: invalid syntax (<ipython-input-8-53bd94ff87c3>, line 3)

![Alt Text](https://media.giphy.com/media/vFKqnCdLPNOKc/giphy.gif)